In [82]:
import os, glob
import pandas as pd
import subprocess as sp
from IPython.core import display as ICD 
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 159)


In [83]:
scan_file ="bbx_quality_report_11.19.tsv"

In [84]:
scan_df = pd.read_csv(scan_file,sep="\t")

In [85]:
scan_df.columns.values

array(['subject', 'BIDS check', 'fmriprep check', 'Anat', 'epi_r1',
       'sdc_r1', 'epi_r2', 'sdc_r2', 'epi_r3', 'sdc_r3', 'epi_r4',
       'sdc_r4', 'From TSV Files', 'fd_r1', 'min_r1', 'max_r1',
       'mcflirt_3mm_r1', 'fd_r2', 'min_r2', 'min_r2.1', 'mcflirt_3mm_r2',
       'fd_r3', 'min_r3', 'max_r3', 'mcflirt_3mm_r3', 'fd_r4', 'min_r4',
       'max_r4', 'mcflirt_3mm_r4', 'FSL Level 1', 'activation_r1',
       'activation_r2', 'activation_r3', 'activation_r4', 'FINAL CHECK',
       'Intials'], dtype=object)

In [86]:
scan_df = scan_df.set_index('subject')
scan_df.tail()

,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,sdc_r4,From TSV Files,fd_r1,min_r1,max_r1,mcflirt_3mm_r1,fd_r2,min_r2,min_r2.1,mcflirt_3mm_r2,fd_r3,min_r3,max_r3,mcflirt_3mm_r3,fd_r4,min_r4,max_r4,mcflirt_3mm_r4,FSL Level 1,activation_r1,activation_r2,activation_r3,activation_r4,FINAL CHECK,Intials
subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sub-153,y,--,good,good,good,good,good,good,good,good,good,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-154,y,--,good,good,missing,good,missing,good,missing,good,missing,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-156,y,--,good,good,missing,good,missing,good,missing,good,missing,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-157,y,--,good,good,good,good,good,good,good,good,good,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-159,y,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
bids_path = '/projects/niblab/bids_projects/Experiments/bbx/bids'
# FD check
def write_report(sub_id, run_id, comparator, fd_total_ct, nifti):
    print("sub: {} \trun: {}  \tcomparator: {} \tfd_count: {} \nNifti: {}".format(sub_id,
                                         run_id, comparator, fd_total_ct, nifti.split("/")[-1]))
    
sub_ids = [x.split("/")[-2] for x in glob.glob(os.path.join(bids_path, 'derivatives/fmriprep/sub-*/ses-1'))]
for sub_id in sub_ids:
    tsv_files=glob.glob(os.path.join(bids_path, 'derivatives/fmriprep/{}/ses-1/func/sub-*_ses-1_task-training_run-*_desc-confounds_regressors.tsv'.format(sub_id)))
    
    for tsv in sorted(tsv_files):
        fd_total_ct = 0
        sub_id=tsv.split("/")[-1].split("_")[0]
        #print(tsv)
        run_id=tsv.split("_")[4]
        nifti=os.path.join(bids_path,
                           'derivatives/fmriprep/{}/ses-1/func/{}_ses-1_task-training_{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz '.format(sub_id, sub_id, run_id))
        shell_cmd1 = 'fslnvols {}'.format(nifti) 
        volume = sp.check_output(shell_cmd1, shell=True, encoding="utf-8")
        volume = volume.strip()
        comparator = int(volume) *.25
        temp_df = pd.read_csv(tsv, sep="\t")
        
        body_motcor_params = temp_df[['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z' ]]
        #print(body_motcor_params)
        try:
            
            spike_regressors = temp_df[['motion_outlier00','motion_outlier01', 'motion_outlier02', 'motion_outlier03','motion_outlier04', 'motion_outlier05']]
            #print(spike_regressors)
        except:
            # add report here
            print("Not found for " + sub_id +" " +run_id)

        max_fd = temp_df['framewise_displacement'].max()
        min_fd = temp_df['framewise_displacement'].min()
        
    
        for fd_val in  temp_df['framewise_displacement']:
            if fd_val > 0.9:
                #print("bad fd value, ", fd_val)
                fd_total_ct += 1
        
       
        if run_id == "run-1":
            scan_df.loc[sub_id, "min_r1"]=min_fd
            scan_df.loc[sub_id, "max_r1"]=max_fd
            
            if max_fd < 3.0:
                scan_df.loc[sub_id, "mcflirt_3mm_r1"]="pass"
            else:
                scan_df.loc[sub_id, "mcflirt_3mm_r1"]="fail"
                
            if fd_total_ct < comparator:
                scan_df.loc[sub_id, "fd_r1"]="pass"
            else:
                scan_df.loc[sub_id, "fd_r1"]="fail"
                write_report(sub_id, run_id, comparator, fd_total_ct, nifti)
                
        elif run_id == "run-2":
            scan_df.loc[sub_id, "min_r2"]=min_fd
            scan_df.loc[sub_id, "max_r2"]=max_fd
            
            if max_fd < 3.0:
                scan_df.loc[sub_id, "mcflirt_3mm_r2"]="pass"
            else:
                scan_df.loc[sub_id, "mcflirt_3mm_r2"]="fail"
                
            if fd_total_ct < comparator:
                scan_df.loc[sub_id, "fd_r2"]="pass"
            else:
                scan_df.loc[sub_id, "fd_r2"]="fail"
                write_report(sub_id, run_id, comparator, fd_total_ct, nifti)
                
        elif run_id == "run-3":
            scan_df.loc[sub_id, "min_r3"]=min_fd
            scan_df.loc[sub_id, "max_r3"]=max_fd
            
            if max_fd < 3.0:
                scan_df.loc[sub_id, "mcflirt_3mm_r3"]="pass"
            else:
                scan_df.loc[sub_id, "mcflirt_3mm_r3"]="fail"
                
                
            if fd_total_ct < comparator:
                scan_df.loc[sub_id, "fd_r3"]="pass"
            else:
                scan_df.loc[sub_id, "fd_r3"]="fail"
                write_report(sub_id, run_id, comparator, fd_total_ct, nifti)
                
        else:
            scan_df.loc[sub_id, "min_r4"]=min_fd
            scan_df.loc[sub_id, "max_r4"]=max_fd
            
            if max_fd < 3.0:
                scan_df.loc[sub_id, "mcflirt_3mm_r4"]="pass"
            else:
                scan_df.loc[sub_id, "mcflirt_3mm_r4"]="fail"
                
                
            if fd_total_ct < comparator:
                scan_df.loc[sub_id, "fd_r4"]="pass"
            else:
                scan_df.loc[sub_id, "fd_r4"]="fail"
                write_report(sub_id, run_id, comparator, fd_total_ct, nifti)
            
        


sub: sub-070 	run: run-1  	comparator: 58.25 	fd_count: 72 
Nifti: sub-070_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
Not found for sub-058 run-1
Not found for sub-078 run-3
sub: sub-100 	run: run-1  	comparator: 0.0 	fd_count: 1 
Nifti: sub-100_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-2  	comparator: 0.0 	fd_count: 0 
Nifti: sub-100_ses-1_task-training_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-3  	comparator: 0.0 	fd_count: 0 
Nifti: sub-100_ses-1_task-training_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-4  	comparator: 0.0 	fd_count: 2 
Nifti: sub-100_ses-1_task-training_run-4_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-098 	run: run-1  	comparator: 0.0 	fd_count: 0 
Nifti: sub-098_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-098 	run: run-2  	comparator: 0.

In [89]:
scan_df

,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,sdc_r4,From TSV Files,fd_r1,min_r1,max_r1,mcflirt_3mm_r1,fd_r2,min_r2,min_r2.1,mcflirt_3mm_r2,fd_r3,min_r3,max_r3,mcflirt_3mm_r3,fd_r4,min_r4,max_r4,mcflirt_3mm_r4,FSL Level 1,activation_r1,activation_r2,activation_r3,activation_r4,FINAL CHECK,Intials,max_r2
subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sub-001,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.038943,0.937265,pass,pass,0.031288,NaN,pass,pass,0.021486,0.864922,pass,pass,0.036686,1.315115,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,1.492913
sub-002,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.029952,0.582735,pass,pass,0.033639,NaN,pass,pass,0.015229,0.491711,pass,pass,0.041908,0.701858,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,0.349757
sub-003,y,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.021921,1.766394,pass,pass,0.028898,NaN,pass,pass,0.026123,1.388662,pass,pass,0.024134,0.881343,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,1.020489
sub-005,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.008253,0.590167,pass,pass,0.011320,NaN,pass,pass,0.005587,0.600573,pass,pass,0.021201,0.675988,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,0.927735
sub-006,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.070449,1.335473,pass,pass,0.029588,NaN,pass,pass,0.032128,7.621521,fail,pass,0.029703,2.800106,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,1.903956
sub-007,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.049019,0.762822,pass,pass,0.033553,NaN,pass,pass,0.042508,0.694882,pass,pass,0.036950,0.798493,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,0.559331
sub-008,y,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-009,y,--,good,good,good,good,good,good,good,good,good,--,pass,0.023569,0.449336,pass,pass,0.036105,NaN,pass,pass,0.016274,0.490951,pass,pass,0.032103,0.400754,pass,--,NaN,NaN,NaN,NaN,NaN,NaN,0.524103


In [90]:
scan_df.to_csv("bbx_qc_11.25.19.tsv", sep="\t")